In [15]:
#                                                   LIBRARIES

# Image preprocesssing- processing
import cv2
from PIL import Image
import numpy as np

# Neural Network libraries
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

In [16]:
num_classes = 3
train_animals_reduced = np.load("2_train_animals_reduced.npy")
train_labels_reduced = np.load("2_train_labels_reduced.npy")
test_animals_reduced = np.load("2_test_animals_reduced.npy")
test_labels_reduced = np.load("2_test_labels_reduced.npy")

In [17]:
# One hot encoding for labels
train_labels_reduced=keras.utils.to_categorical(train_labels_reduced,num_classes)
test_labels_reduced=keras.utils.to_categorical(test_labels_reduced,num_classes)

In [18]:
# import sequential model and all the required layers
def create_model():
    #make model
    model=Sequential()
    # Pairs of Conv2D layer and MaxPool2D Layer with increasing filter sizes ( 16,32 ,64). This helps to make image grow more in depthwise and become more flatten.
    # Maxpool: great as they optimize the training time

    '''# Pair 1 (16)
    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
    model.add(MaxPooling2D(pool_size=2))

    # Pair 2 (32)
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))

    # Pair 3 (64)
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))

    # Dropout layers to reduce overfitting
    model.add(Dropout(0.2))
    model.add(Flatten())


    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))'''

    # capas de calculo -> ir de menor a mayor, suelen ser potencias / multiplos de 2 (las layers)

    model.add(Conv2D(filters=64,kernel_size=3,padding="same",activation="relu",input_shape=(50,50,3))) # parte imp = relu
    model.add(MaxPooling2D(pool_size=2))

    model.add(Conv2D(filters=128,kernel_size=3,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))

    model.add(Conv2D(filters=256,kernel_size=3,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=4))

    model.add(Dropout(0.2))
    model.add(Flatten()) # without flatten output shape = (50, 50, 6) -> flatten = (None, 6) which we need to get layer output


    # capas de clasificacion

    model.add(Dense(200,activation="relu"))

    # Final dense layer with num_classes nodes = categories of animals we have in the set
    # Softmax activation is used to give scores to these categories which lie between 0 and 1.
    model.add(Dense(num_classes, activation="softmax"))
    model.summary()

    # compile the model
    # We use loss function as categorical_crossentropy and Adam optimizer

    # if binary data -> loss = Binary Cross Entropy and activation = sigmoid
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                      metrics=['accuracy'])

    return model

In [19]:
model = create_model()
model.load_weights('model.h5')

loss, acc = model.evaluate(test_animals_reduced, test_labels_reduced, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 50, 50, 64)        1792      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 25, 25, 64)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 25, 25, 128)       73856     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 12, 12, 128)      0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 12, 12, 256)       295168    
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 3, 3, 256)       

In [20]:
model.fit(train_animals_reduced,train_labels_reduced,batch_size=32 ,epochs=30,verbose=1)

Epoch 1/30
32/32 [==============================] - 4s 97ms/step - loss: 1.2525 - accuracy: 0.4990
Epoch 2/30
32/32 [==============================] - 3s 97ms/step - loss: 0.9165 - accuracy: 0.5891
Epoch 3/30
32/32 [==============================] - 3s 98ms/step - loss: 0.8343 - accuracy: 0.6307
Epoch 4/30
32/32 [==============================] - 3s 97ms/step - loss: 0.7513 - accuracy: 0.6594
Epoch 5/30
32/32 [==============================] - 3s 93ms/step - loss: 0.6840 - accuracy: 0.7000
Epoch 6/30
32/32 [==============================] - 3s 92ms/step - loss: 0.5415 - accuracy: 0.7723
Epoch 7/30
32/32 [==============================] - 3s 94ms/step - loss: 0.5700 - accuracy: 0.7416
Epoch 8/30
32/32 [==============================] - 3s 95ms/step - loss: 0.4897 - accuracy: 0.8040
Epoch 9/30
32/32 [==============================] - 3s 92ms/step - loss: 0.4084 - accuracy: 0.8307
Epoch 10/30
32/32 [==============================] - 3s 93ms/step - loss: 0.3372 - accuracy: 0.8644
Epoch 11/

In [21]:
# test the model
score = model.evaluate(test_animals_reduced, test_labels_reduced, verbose=1)
print('\n', 'Test accuracy:', score[1])

8/8 [==============================] - 0s 26ms/step - loss: 2.0543 - accuracy: 0.6478

 Test accuracy: 0.647826075553894


In [22]:
def convert_to_array(img):
    im = cv2.imread(img)
    img = Image.fromarray(im, 'RGB')
    image = img.resize((50, 50))
    return np.array(image)

def get_animal_name(label):
    if label==0:
        return "Bear"
    if label==1:
        return "Chicken"
    if label==2:
        return "Turtle"

def predict_animal(file):
    print("Predicting .................................")
    ar=convert_to_array(file)
    ar=ar/255
    label=1
    a=[]
    a.append(ar)
    a=np.array(a)
    score=model.predict(a,verbose=1)
    print(score)
    label_index=np.argmax(score)
    print(label_index)
    acc=np.max(score)
    animal=get_animal_name(label_index)
    print(animal)
    print("The predicted Animal is a "+animal+" with accuracy =    "+str(acc))

In [25]:
# predict image
file = "../datasets/animals/animals-detection-images-dataset/prediction/bear.jpg"
predict_animal(file)

Predicting .................................
1/1 [==============================] - 0s 21ms/step
[[0.25310296 0.32158893 0.4253081 ]]
2
Turtle
The predicted Animal is a Turtle with accuracy =    0.4253081
